Perfect 🚑 — let’s adapt the **hospital triage agent notebook** to work properly with **`phi3:mini`**.  

Because `phi3:mini` is small and **not great at strict JSON output**, we’ll:  
1. Use the **simpler `zero-shot-react-description` agent type** (text‑based ReAct).  
2. Add **very explicit tool descriptions** so it calls tools correctly.  
3. Enable **`handle_parsing_errors=True`** so the agent won’t silently die if its format is a little off.  
4. Make the tool functions more forgiving (accept empty input by default).  

This way, you can still demo the workflow with `phi3:mini`.

---

# 🏥 Jupyter Notebook: Hospital Triage Assistant with **phi3:mini**

---

```markdown
# 🏥 Hospital Triage Assistant using phi3:mini

---

## 📌 Overview

This notebook shows how a **small local LLM (phi3:mini)** can act as an **Agentic AI triage assistant**:

- Retrieve patient vitals (from a Python function).
- Search urgent care guidelines (via DuckDuckGo).
- Summarize condition.
- Save a triage report.

⚠️ Note: `phi3:mini` is a *tiny model*. We simplify prompts and parsing for reliability.
```

---

## 1. Setup

```python
!pip install langchain langchain-community duckduckgo-search pandas
```

---

## 2. Initialize Ollama with phi3:mini

```python
from langchain_community.chat_models import ChatOllama

# Use phi3:mini model via Ollama
llm = ChatOllama(model="phi3:mini", temperature=0)

# Quick test
resp = llm.invoke("Write a short encouraging message for a nurse on duty.")
print("🧠 phi3:mini says:", resp.content)
```

---

## 3. Define Tools

```python
import pandas as pd
from pathlib import Path
from langchain.agents import Tool
from langchain_community.tools import DuckDuckGoSearchRun

# --- Tool 1: Guideline search ---
search = DuckDuckGoSearchRun()

# --- Tool 2: Patient vitals (mock data) ---
def load_patient_vitals(_=""):
    vitals = {
        "Heart Rate (bpm)": 110,
        "Blood Pressure (mmHg)": "145/95",
        "Temperature (C)": 38.5,
        "Respiratory Rate": 24,
        "Oxygen Saturation (%)": 93
    }
    return pd.DataFrame([vitals]).to_string(index=False)

# --- Tool 3: Save report ---
def save_triage_report(report: str):
    path = Path("triage_report.txt")
    with open(path, "w") as f:
        f.write(report)
    return f"📄 Report saved at {path.absolute()}"


# --- Toolbox with strong instructions ---
tools = [
    Tool(
        name="MedicalGuidelineSearch",
        func=search.run,
        description=(
            "Use this to look up urgent care protocols or guidelines. "
            "Example correct call:\n"
            "Action: MedicalGuidelineSearch\n"
            "Action Input: chest pain urgent care protocol"
        )
    ),
    Tool(
        name="PatientVitals",
        func=load_patient_vitals,
        description=(
            "Fetch current patient vitals. "
            "IMPORTANT: Must call exactly as:\n"
            "Action: PatientVitals\n"
            "Action Input: ''"
        )
    ),
    Tool(
        name="SaveTriageReport",
        func=save_triage_report,
        description=(
            "Save the final triage report into a text file. "
            "Correct format is:\n"
            "Action: SaveTriageReport\n"
            "Action Input: <the report text>"
        )
    )
]
```

---

## 4. Build the Agent

```python
from langchain.agents import initialize_agent

# Simpler text-based agent (phi3-friendly)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True  # retry if format is wrong
)
```

---

## 5. Run Triage Simulation

```python
task = """
A man arrived at the ER with chest pain and difficulty breathing.
1. Retrieve his vitals.
2. If abnormal, search for urgent care guidelines.
3. Summarize condition with risks.
4. Save the triage report into a file.
"""

result = agent.run(task)

print("\n✅ Final Agent Output:\n", result)
```

---

## 6. Inspect Saved Report

```python
with open("triage_report.txt", "r") as f:
    print("\n📋 TRIAGE REPORT CONTENT 📋\n")
    print(f.read())
```

---

## 7. Expected Flow with phi3:mini  

With `verbose=True`, you’ll see **ReAct cycle logs** like:

```
> Entering new AgentExecutor chain...
Question: A man arrived at the ER...
Thought: I should first fetch his vitals.
Action: PatientVitals
Action Input: 
Observation:
 Heart Rate (bpm)     110
 Blood Pressure(mmHg) 145/95
 Temp (C)             38.5
 Resp Rate            24
 SpO2                 93
Thought: These vitals are abnormal (tachycardia, hypertension, fever, mild hypoxia). I should search guidelines.
Action: MedicalGuidelineSearch
Action Input: chest pain breathlessness urgent care
Observation: [short search results...]
Thought: I'll now summarize in a triage report.
Action: SaveTriageReport
Action Input: "Patient vitals show... urgent cardiology consult advised."
Observation: 📄 Report saved at triage_report.txt
Final Answer: The triage report has been saved.
```

---

# 🔑 Why This Works Better for `phi3:mini`

- We use **`zero-shot-react-description`** → simpler (no JSON parsing).  
- **Explicit tool instructions** → guides phi3:mini away from brackets/markdown errors.  
- **`handle_parsing_errors=True`** → recovers gracefully when format is imperfect.  
- Tools tolerate empty input.  

---

# ✅ Final Takeaway  

- **phi3:mini** can *simulate* this workflow if you simplify prompts, relax parsing, and give very clear tool usage instructions.  
- For more robust results (smoother tool calls, complex workflows), switch to a larger local model like **`llama3:8b`** or **`mistral:7b-instruct`**.  

---

👉 Would you like me to now **extend this phi3:mini notebook into a *multi-agent hospital setup*** (e.g. `NurseAgent` → gets vitals, `DoctorAgent` → guidelines, `ClerkAgent` → saves report), so you can see **collaboration even with a small model**?